In [1]:
# 필요한 라이브러리 Import
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables import RunnablePassthrough
import time
from datetime import datetime

In [2]:
# 프로그래밍 언어 정의
PROGRAMMING_LANGUAGES = {
    "python": {
        "name": "Python",
        "description": "간단하고 읽기 쉬운 고수준 프로그래밍 언어"
    },
    "java": {
        "name": "Java",
        "description": "객체지향 프로그래밍을 지원하는 범용 프로그래밍 언어"
    },
    "javascript": {
        "name": "JavaScript",
        "description": "웹 개발에 주로 사용되는 스크립트 언어"
    },
    "cpp": {
        "name": "C++",
        "description": "고성능 시스템 프로그래밍을 위한 언어"
    },
    "go": {
        "name": "Go",
        "description": "Google이 개발한 현대적인 시스템 프로그래밍 언어"
    },
    "rust": {
        "name": "Rust",
        "description": "안전성과 성능을 모두 갖춘 시스템 프로그래밍 언어"
    },
    "swift": {
        "name": "Swift",
        "description": "Apple 생태계를 위한 현대적인 프로그래밍 언어"
    },
    "kotlin": {
        "name": "Kotlin",
        "description": "JVM 기반의 현대적인 프로그래밍 언어"
    }
}

In [3]:
# 프롬프트 템플릿 정의
poetry_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 프로그래밍 언어의 특징과 철학을 시로 표현하는 뛰어난 시인입니다.
    각 언어의 다음 특징들을 반드시 포함해주세요:
    - 주요 문법적 특징
    - 대표적인 철학/패러다임
    - 실제 사용되는 대표적인 분야
    - 해당 언어만의 독특한 특징
    """),
    ("user", "다음 프로그래밍 언어에 대한 시를 써주세요: {language}")
])

explanation_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 시의 의미를 깊이 있게 해석하고 설명하는 문학 평론가입니다.
    다음 구조로 해석을 제공해주세요:
    1. 시의 전체적인 주제와 의도
    2. 각 연별 상세 해석
    3. 사용된 문학적 장치(은유, 직유 등) 분석
    4. 프로그래밍 언어의 특징과의 연관성
    5. 결론
    """),
    ("user", "다음 시의 의미와 상징, 프로그래밍 언어와의 연관성을 설명해주세요:\n{poem}")
])

In [4]:
# 모델과 체인 설정
model = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

poetry_chain = poetry_prompt | model | StrOutputParser()

final_chain = (
    RunnablePassthrough()
    | {"language": lambda x: x["language"]}
    | {"poem": poetry_chain, "language": lambda x: x["language"]}
    | {
        "poem": lambda x: x["poem"],
        "explanation": explanation_prompt | model | StrOutputParser()
    }
)

In [5]:
# 유틸리티 함수들 정의
def ensure_directory():
    """day01 디렉토리가 존재하지 않으면 생성"""
    if not os.path.exists('day01'):
        os.makedirs('day01')

def save_poetry(language: str, poem: str, explanation: str):
    """시와 해석을 파일로 저장"""
    ensure_directory()
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"day01/{language.lower()}_{timestamp}.txt"
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"=== {language}에 대한 시 ===\n\n")
        f.write(poem)
        f.write("\n\n=== 해석 ===\n\n")
        f.write(explanation)
    
    return filename

def generate_single_poetry(language: str):
    """단일 언어에 대한 시 생성"""
    print(f"\n{language}에 대한 시와 해석을 생성합니다...")
    result = final_chain.invoke({"language": language})
    
    filename = save_poetry(language, result["poem"], result["explanation"])
    
    print(f"\n=== {language}에 대한 시 ===")
    print(result["poem"])
    print("\n=== 해석 ===")
    print(result["explanation"])
    print(f"\n저장된 파일: {filename}")
    
    return result

def generate_all_poetry():
    """모든 프로그래밍 언어에 대한 시 생성"""
    results = {}
    
    for key, lang_info in PROGRAMMING_LANGUAGES.items():
        language = lang_info["name"]
        print(f"\n{language} 처리 중...")
        
        result = generate_single_poetry(language)
        results[language] = result
        
        # API 호출 사이에 잠시 대기
        time.sleep(2)
    
    return results

def show_menu():
    """메인 메뉴 표시"""
    print("\n=== 프로그래밍 언어 시 생성기 ===")
    print("1. 단일 언어 선택")
    print("2. 모든 언어")
    print("3. 종료")
    return input("선택해주세요 (1-3): ")

def show_language_menu():
    """언어 선택 메뉴 표시"""
    print("\n=== 사용 가능한 프로그래밍 언어 ===")
    for idx, (key, lang) in enumerate(PROGRAMMING_LANGUAGES.items(), 1):
        print(f"{idx}. {lang['name']} - {lang['description']}")
    
    while True:
        selection = input("\n언어를 선택하세요 (1-8): ")
        try:
            idx = int(selection)
            if 1 <= idx <= len(PROGRAMMING_LANGUAGES):
                return list(PROGRAMMING_LANGUAGES.values())[idx-1]['name']
        except ValueError:
            pass
        print("올바른 번호를 입력해주세요.")

In [6]:
# 실행
def main():
    """메인 실행 함수"""
    while True:
        choice = show_menu()
        
        if choice == "1":
            language = show_language_menu()
            generate_single_poetry(language)
        elif choice == "2":
            generate_all_poetry()
        elif choice == "3":
            print("프로그램을 종료합니다.")
            break
        else:
            print("올바른 옵션을 선택해주세요.")

if __name__ == "__main__":
    main()


=== 프로그래밍 언어 시 생성기 ===
1. 단일 언어 선택
2. 모든 언어
3. 종료

Python 처리 중...

Python에 대한 시와 해석을 생성합니다...
**Python의 노래**

들꽃처럼 자유로운, 파이썬의 언어,  
들여다보면, 간결함의 미소가 보인다.  
중괄호 대신, 들여쓰기로 맺은,  
가독성의 미학, 프로그래머의 꿈을 담아.

객체지향과 절차적 흐름이 조화를 이루며,  
모든 것을 하나로 묶는, 유연한 철학,  
데이터 과학과 웹의 세계를 탐험하며,  
인공지능의 바다에서, 그 깊이를 헤엄친다.

모듈과 라이브러리, 무한한 가능성의 길,  
간단한 코드로 복잡함을 해결하는 기적,  
‘Hello, World!’의 첫 걸음,  
초보자도 전문가처럼, 빠르게 나아가리.

파이썬, 너는 강력함과 단순함의 연주,  
대화하듯, 손끝의 흐름을 이끄는,  
코드의 미로에서 길을 잃지 않게 해주리,  
지속 가능한 미래를 향한, 그 고요한 희망의 소리.### 1. 시의 전체적인 주제와 의도

이 시는 프로그래밍 언어인 파이썬(Python)에 대한 찬가로, 그 특징과 장점을 아름답고 감성적으로 표현하고 있습니다. 주제는 프로그래밍의 복잡성을 간단하게 만들어주는 파이썬의 자유롭고 우아한 특성에 중점을 두고 있으며, 이를 통해 프로그래머들에게 영감을 주고, 프로그래밍 언어의 매력을 전달하려는 의도가 담겨 있습니다. 또한, 파이썬이 다양한 분야에서 활용될 수 있는 가능성을 강조하고 있으며, 초보자부터 전문가에 이르기까지 모두에게 열려 있는 기회를 상징적으로 나타냅니다.

### 2. 각 연별 상세 해석

- **1연:** 첫 연에서는 파이썬의 간결함과 가독성을 강조합니다. "들꽃처럼 자유로운"이라는 표현은 자연의 자유로움과 프로그래밍 언어의 유연함을 연결지어, 파이썬이 가진 미적 요소를 부각시킵니다. 중괄호 대신 들여쓰기를 사용함으로써 더 간결하고 명확한 코드를 작성할 수 있다는 점도 강조됩니다.
  
- **2연:** 두 번째 연에서는 파이썬의 객체